# Implementación Base del Algoritmo IOSRCRVD mediante Optimización por Colonia de Hormigas

## Introducción

En este trabajo se implementa un algoritmo basado en **Optimización por Colonia de Hormigas (Ant Colony Optimization, ACO)** para abordar el problema IOSRCRVD, el cual surge en contextos de respuesta ante desastres naturales y gestión de infraestructuras críticas. El problema considera de manera integrada la **planificación de reparaciones de carreteras dañadas** y la **distribución de ayuda humanitaria**, teniendo en cuenta que el acceso a determinados nodos depende del momento en que las reparaciones han sido completadas.

El objetivo principal es minimizar el **tiempo de finalización del último servicio de ayuda**, coordinando de forma eficiente ambos procesos bajo restricciones temporales y de conectividad.

---

## Descripción general del código base

El código base ha sido diseñado con un enfoque modular y reutilizable, de manera que el núcleo del algoritmo es independiente de los datos concretos de cada escenario. Esto permite aplicar el mismo procedimiento de optimización a distintos grafos y configuraciones sin modificar la lógica principal.

La implementación se apoya en tres ideas fundamentales:

1. La representación explícita del problema mediante estructuras de datos claras.
2. El uso de ACO como metaheurística para construir soluciones de forma probabilística.
3. La incorporación de dependencias temporales a través de un modelo de espera en los nodos de reparación.

---

## Representación del problema

El escenario se modela como un grafo no dirigido en el que los arcos representan tiempos de viaje. A partir de este grafo se distinguen dos subconjuntos de nodos:

- **Nodos de reparación**, que deben ser intervenidos antes de poder ser utilizados sin restricciones.
- **Nodos de demanda**, que requieren la prestación de un servicio con una duración determinada.

Cada nodo de reparación tiene asociado un tiempo de reparación, mientras que cada nodo de demanda tiene un tiempo de servicio. Ambos procesos parten de un mismo depósito inicial.

Toda esta información se encapsula en una única estructura de datos, lo que facilita la definición de distintos escenarios de prueba sin alterar el algoritmo.

---

## Lógica general del algoritmo ACO

El algoritmo sigue el esquema clásico de ACO, adaptado a la naturaleza dual del problema. En cada iteración, un conjunto de hormigas construye soluciones completas en dos fases consecutivas:

1. **Construcción de la ruta de reparación**, donde se decide el orden en el que el equipo de reparación visita los nodos dañados.
2. **Construcción de la ruta de ayuda humanitaria**, donde el vehículo de ayuda visita los nodos de demanda respetando las dependencias impuestas por las reparaciones.

Las decisiones de cada hormiga se toman mediante una regla probabilística que combina información de feromonas con una heurística basada en el coste temporal del desplazamiento.

---

## Gestión de las dependencias temporales

Uno de los aspectos más relevantes del código base es la forma en que se gestionan las restricciones temporales. Para ello, se utiliza un algoritmo de caminos mínimos que calcula el **tiempo de llegada más temprano** a cada nodo.

Cuando el vehículo de ayuda alcanza un nodo de reparación cuya intervención aún no ha finalizado, el modelo introduce automáticamente un tiempo de espera hasta que la reparación se completa. De esta forma, el algoritmo refleja fielmente la interdependencia entre ambas operaciones sin necesidad de imponer ventanas de tiempo de manera explícita.

---

## Función objetivo y aprendizaje mediante feromonas

La calidad de una solución se mide mediante el tiempo de finalización del último servicio de ayuda. Este valor actúa como función objetivo y se utiliza para actualizar las feromonas del sistema.

Tras cada iteración:

- Las feromonas se evaporan para evitar convergencia prematura.
- La mejor solución encontrada refuerza las transiciones utilizadas, favoreciendo que estas decisiones se repitan en iteraciones futuras.

Este mecanismo permite un equilibrio entre exploración del espacio de soluciones y explotación de las mejores rutas encontradas.

---

## Utilidad práctica y extensibilidad

Desde un punto de vista práctico, el código base permite analizar de forma integrada decisiones de reparación y distribución bajo incertidumbre y restricciones temporales. Su diseño facilita la reproducibilidad de los experimentos y la comparación entre distintos escenarios.

Además, la estructura modular del código permite extender el modelo de manera natural, por ejemplo incorporando múltiples vehículos, prioridades en los nodos de demanda o variantes más complejas del algoritmo ACO.

---

En las siguientes secciones se presentarán distintos ejemplos de aplicación del código base, mostrando cómo el algoritmo se adapta a diferentes escenarios y analizando las soluciones obtenidas en cada caso.


In [3]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Set
import math
import random
import heapq

Node = int


# ======================================================
# Datos del problema
# ======================================================

@dataclass
class ProblemData:
    adj: Dict[Node, List[Tuple[Node, float]]]
    depot: Node
    repair_nodes: Set[Node]
    demand_nodes: Set[Node]
    repair_time: Dict[Node, float]          # s_i
    demand_service_time: Dict[Node, float]  # s'_i


@dataclass
class ACOParams:
    alpha: float = 1.0
    beta: float = 2.0
    rho: float = 0.2
    Q: float = 1.0
    num_ants: int = 60
    num_iters: int = 300
    seed: int = 7


@dataclass
class Solution:
    repair_route: List[Node]
    relief_route: List[Node]
    repair_done_time: Dict[Node, float]     # d_i
    objective: float                        # fin del último servicio


# ======================================================
# Utilidades de grafo
# ======================================================

def add_undirected_edge(adj: Dict[Node, List[Tuple[Node, float]]], u: Node, v: Node, w: float):
    adj.setdefault(u, []).append((v, w))
    adj.setdefault(v, []).append((u, w))


def dijkstra_static(adj: Dict[Node, List[Tuple[Node, float]]], source: Node) -> Dict[Node, float]:
    dist: Dict[Node, float] = {source: 0.0}
    pq = [(0.0, source)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist.get(u, math.inf):
            continue
        for v, w in adj.get(u, []):
            nd = d + w
            if nd < dist.get(v, math.inf):
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return dist


def earliest_arrival_dijkstra(
    adj: Dict[Node, List[Tuple[Node, float]]],
    source: Node,
    start_time: float,
    repair_done: Dict[Node, float],
    repair_nodes: Set[Node],
) -> Dict[Node, float]:
    """
    Dijkstra FIFO de llegada más temprana con espera:
    si llegas a un nodo de reparación antes de d_i, esperas hasta d_i.
    """
    dist: Dict[Node, float] = {source: start_time}
    pq = [(start_time, source)]

    def wait_if_needed(node: Node, t: float) -> float:
        if node in repair_nodes:
            return max(t, repair_done.get(node, 0.0))
        return t

    dist[source] = wait_if_needed(source, dist[source])

    while pq:
        t_u, u = heapq.heappop(pq)
        if t_u != dist.get(u, math.inf):
            continue
        for v, w in adj.get(u, []):
            t_v = wait_if_needed(v, t_u + w)
            if t_v < dist.get(v, math.inf):
                dist[v] = t_v
                heapq.heappush(pq, (t_v, v))
    return dist


def roulette_choice(cands: List[Node], weights: List[float]) -> Node:
    s = sum(weights)
    if s <= 0:
        return random.choice(cands)
    r = random.random() * s
    acc = 0.0
    for c, w in zip(cands, weights):
        acc += w
        if acc >= r:
            return c
    return cands[-1]


# ======================================================
# ACO (estructura común)
# ======================================================

class IOSRCRVD_ACO:
    def __init__(self, data: ProblemData, p: ACOParams):
        self.data = data
        self.p = p
        random.seed(p.seed)

        self.tau_rep: Dict[Tuple[Node, Node], float] = {}
        self.tau_dem: Dict[Tuple[Node, Node], float] = {}
        self.static_dist_cache: Dict[Node, Dict[Node, float]] = {}

        self._init_pheromones()

    def _init_pheromones(self):
        init = 1.0
        rep_nodes = self.data.repair_nodes | {self.data.depot}
        dem_nodes = self.data.demand_nodes | {self.data.depot}

        for i in rep_nodes:
            for j in rep_nodes:
                if i != j:
                    self.tau_rep[(i, j)] = init

        for i in dem_nodes:
            for j in dem_nodes:
                if i != j:
                    self.tau_dem[(i, j)] = init

    def _heur(self, cost: float) -> float:
        return 1.0 / max(cost, 1e-6)

    def _static_dist(self, src: Node) -> Dict[Node, float]:
        if src not in self.static_dist_cache:
            self.static_dist_cache[src] = dijkstra_static(self.data.adj, src)
        return self.static_dist_cache[src]

    # ---------------- Repair crew ----------------

    def build_repair(self) -> Tuple[List[Node], Dict[Node, float]]:
        route = [self.data.depot]
        remaining = set(self.data.repair_nodes)

        t = 0.0
        current = self.data.depot
        done: Dict[Node, float] = {}

        while remaining:
            dist = self._static_dist(current)
            cands = [v for v in remaining if v in dist and math.isfinite(dist[v])]
            if not cands:
                return route, {v: math.inf for v in self.data.repair_nodes}

            weights = []
            for v in cands:
                weights.append(
                    (self.tau_rep.get((current, v), 1.0) ** self.p.alpha) *
                    (self._heur(dist[v]) ** self.p.beta)
                )

            nxt = roulette_choice(cands, weights)

            t += dist[nxt]
            t += self.data.repair_time.get(nxt, 0.0)
            done[nxt] = t

            route.append(nxt)
            remaining.remove(nxt)
            current = nxt

        return route, done

    # ---------------- Relief vehicle ----------------

    def build_relief(self, repair_done: Dict[Node, float]) -> Tuple[List[Node], float]:
        route = [self.data.depot]
        remaining = set(self.data.demand_nodes)

        t = 0.0
        current = self.data.depot
        last_finish = 0.0

        while remaining:
            dist = earliest_arrival_dijkstra(
                self.data.adj, current, t, repair_done, self.data.repair_nodes
            )
            cands = [v for v in remaining if v in dist and math.isfinite(dist[v])]
            if not cands:
                return route, math.inf

            weights = []
            for v in cands:
                travel_effective = max(dist[v] - t, 0.0)
                weights.append(
                    (self.tau_dem.get((current, v), 1.0) ** self.p.alpha) *
                    (self._heur(travel_effective) ** self.p.beta)
                )

            nxt = roulette_choice(cands, weights)

            t = dist[nxt] + self.data.demand_service_time.get(nxt, 0.0)
            last_finish = t

            route.append(nxt)
            remaining.remove(nxt)
            current = nxt

        return route, last_finish

    # ---------------- Pheromones ----------------

    def evaporate(self):
        for k in list(self.tau_rep.keys()):
            self.tau_rep[k] *= (1.0 - self.p.rho)
        for k in list(self.tau_dem.keys()):
            self.tau_dem[k] *= (1.0 - self.p.rho)

    def reinforce(self, sol: Solution):
        if not math.isfinite(sol.objective) or sol.objective <= 0:
            return
        delta = self.p.Q / sol.objective

        for i in range(len(sol.repair_route) - 1):
            a, b = sol.repair_route[i], sol.repair_route[i + 1]
            self.tau_rep[(a, b)] = self.tau_rep.get((a, b), 0.0) + delta

        for i in range(len(sol.relief_route) - 1):
            a, b = sol.relief_route[i], sol.relief_route[i + 1]
            self.tau_dem[(a, b)] = self.tau_dem.get((a, b), 0.0) + delta

    # ---------------- Solve ----------------

    def solve(self) -> Solution:
        best = Solution([], [], {}, math.inf)

        for _ in range(self.p.num_iters):
            iter_best = Solution([], [], {}, math.inf)

            for _k in range(self.p.num_ants):
                rep_route, rep_done = self.build_repair()
                rel_route, obj = self.build_relief(rep_done)
                sol = Solution(rep_route, rel_route, rep_done, obj)

                if sol.objective < iter_best.objective:
                    iter_best = sol

            self.evaporate()
            self.reinforce(iter_best)

            if iter_best.objective < best.objective:
                best = iter_best

        return best


# ======================================================
# Helpers de impresión (opcional)
# ======================================================

def trace_relief(data: ProblemData, sol: Solution):
    """
    Traza paso a paso del vehículo de ayuda:
    origen -> destino | sale | llega (con espera) | servicio | termina
    """
    t = 0.0
    current = data.depot
    rows = []

    for nxt in sol.relief_route[1:]:
        dist = earliest_arrival_dijkstra(data.adj, current, t, sol.repair_done_time, data.repair_nodes)
        arrival = dist[nxt]
        service = data.demand_service_time.get(nxt, 0.0)
        finish = arrival + service

        rows.append((current, nxt, t, arrival, service, finish))

        t = finish
        current = nxt

    return rows


## Ejemplo ilustrativo del funcionamiento del algoritmo IOSRCRVD

## Descripción de los datos y su interpretación práctica

Para ilustrar el funcionamiento del algoritmo IOSRCRVD se considera una instancia pequeña del problema, pensada específicamente para facilitar la comprensión del modelo y de las decisiones que toma el algoritmo. Aunque el tamaño del escenario es reducido, incluye todos los elementos esenciales del problema y reproduce de forma fiel situaciones reales de coordinación entre reparación de infraestructuras y distribución de ayuda.

El escenario se representa mediante un grafo no dirigido, donde los nodos simbolizan distintas ubicaciones geográficas y los arcos representan los tiempos necesarios para desplazarse entre ellas. Estos tiempos de viaje permiten modelar la accesibilidad de la red y condicionan directamente tanto la planificación de las reparaciones como la distribución de ayuda humanitaria.

Todas las operaciones parten de un nodo común que actúa como depósito inicial. Este nodo representa el punto de salida de los recursos disponibles y no está sujeto a reparaciones ni servicios, funcionando únicamente como origen de las rutas.

Dentro del grafo se distinguen dos tipos de nodos con funciones claramente diferenciadas. Por un lado, los nodos de reparación representan infraestructuras dañadas que deben ser intervenidas antes de poder ser utilizadas sin restricciones. En este ejemplo, los nodos 6 y 7 requieren reparaciones con duraciones distintas, lo que refleja diferentes niveles de daño o complejidad técnica. Estos tiempos de reparación determinan el momento a partir del cual la red se vuelve progresivamente más accesible.

Por otro lado, los nodos de demanda representan ubicaciones que requieren la prestación de un servicio, como la entrega de ayuda humanitaria. En el ejemplo, los nodos 3 y 5 requieren tiempos de servicio específicos que modelan la duración necesaria para completar la atención en cada punto. Estos tiempos contribuyen directamente al cálculo del tiempo total de respuesta del sistema.

Un aspecto fundamental del problema es la dependencia temporal entre ambos tipos de nodos. El acceso a los nodos de demanda puede verse condicionado por el estado de las reparaciones, ya que algunos caminos atraviesan nodos aún no reparados. Si el vehículo de ayuda alcanza uno de estos nodos antes de que la reparación haya finalizado, se produce una espera obligatoria hasta que la infraestructura vuelve a estar operativa. Este mecanismo reproduce situaciones reales en las que la distribución de recursos no puede llevarse a cabo hasta que se restablecen ciertas infraestructuras críticas.

A pesar de su simplicidad, esta instancia permite observar claramente cómo los tiempos de reparación influyen en la planificación global y cómo el algoritmo coordina ambas operaciones para minimizar el tiempo de finalización del último servicio. El ejemplo resulta especialmente útil para validar la correcta implementación del modelo, analizar el efecto de las esperas y facilitar la interpretación de los resultados obtenidos en las secciones posteriores.


![Grafo correspondiente al ejemplo pequeño](IMAGENES/image.png)



In [ ]:
def build_example_small() -> ProblemData:
    adj = {}
    add_undirected_edge(adj, 0, 1, 4)
    add_undirected_edge(adj, 1, 6, 2)
    add_undirected_edge(adj, 6, 2, 2)
    add_undirected_edge(adj, 2, 3, 3)
    add_undirected_edge(adj, 2, 7, 2)
    add_undirected_edge(adj, 7, 4, 2)
    add_undirected_edge(adj, 4, 5, 3)

    return ProblemData(
        adj=adj,
        depot=0,
        repair_nodes={6, 7},
        demand_nodes={3, 5},
        repair_time={6: 6.0, 7: 5.0},
        demand_service_time={3: 4.0, 5: 3.0},
    )

data = build_example_small()
params = ACOParams(alpha=1.0, beta=2.0, rho=0.2, Q=1.0, num_ants=50, num_iters=200, seed=7)

aco = IOSRCRVD_ACO(data, params)
best = aco.solve()

print("=== EJEMPLO PEQUEÑO ===")
print("Ruta reparación:", best.repair_route)
print("s_i:", {k: data.repair_time[k] for k in sorted(data.repair_nodes)})
print("d_i:", {k: round(best.repair_done_time[k], 2) for k in sorted(best.repair_done_time)})
print("Ruta ayuda:", best.relief_route)
print("s'_i:", {k: data.demand_service_time[k] for k in sorted(data.demand_nodes)})
print("Objetivo:", round(best.objective, 2))

print("--- Traza ayuda ---")
for a, b, ts, ta, sv, tf in trace_relief(data, best):
    print(f"{a}->{b}: sale {ts:.2f}, llega {ta:.2f}, servicio {sv:.2f}, termina {tf:.2f}")



=== EJEMPLO PEQUEÑO ===
Ruta reparación: [0, 6, 7]
s_i: {6: 6.0, 7: 5.0}
d_i: {6: 12.0, 7: 21.0}
Ruta ayuda: [0, 3, 5]
s'_i: {3: 4.0, 5: 3.0}
Objetivo: 34.0
--- Traza ayuda ---
0->3: sale 0.00, llega 17.00, servicio 4.00, termina 21.00
3->5: sale 21.00, llega 31.00, servicio 3.00, termina 34.00


## Interpretación de la solución obtenida

La ejecución del algoritmo devuelve dos rutas coordinadas: una para el **equipo de reparación** y otra para el **vehículo de ayuda**. La función objetivo es el **instante en el que termina el último servicio de ayuda**; en este ejemplo el objetivo final es **$34.0$**.

### Ruta del equipo de reparación y significado de $d_i$

La ruta de reparación obtenida es:

- Ruta reparación: $[0, 6, 7]$

Esto indica que el equipo sale del depósito (nodo $0$), repara primero el nodo $6$ y después el nodo $7$. Con los tiempos de viaje y las duraciones de reparación $s_i$, se calculan los instantes de finalización $d_i$:

- $d_6 = 12.0$
- $d_7 = 21.0$

Interpretación práctica: $d_i$ representa el tiempo a partir del cual el nodo de reparación $i$ queda completamente operativo. Por tanto:

- En el tiempo $12.0$ el nodo $6$ ya está reparado.
- En el tiempo $21.0$ el nodo $7$ ya está reparado.

Estos valores son fundamentales porque determinan **si el vehículo de ayuda puede atravesar dichos nodos sin esperar**. En términos operativos, esta ruta define el calendario de “reapertura” progresiva de la red.

---

### Ruta del vehículo de ayuda: llegadas, servicio y objetivo

La ruta de ayuda obtenida es:

- Ruta ayuda: $[0, 3, 5]$

La traza temporal calculada por el modelo es:

- $0 \to 3$: sale $0.00$, llega $17.00$, servicio $4.00$, termina $21.00$
- $3 \to 5$: sale $21.00$, llega $31.00$, servicio $3.00$, termina $34.00$

Esto se interpreta así:

1. **Desplazamiento hacia el nodo $3$**  
   El vehículo sale en $t = 0.00$ y llega al nodo $3$ en $t = 17.00$. Este valor corresponde al **tiempo de llegada más temprano**, que incluye automáticamente cualquier posible espera causada por atravesar nodos de reparación aún no disponibles (si aplica en el recorrido óptimo).

2. **Servicio en el nodo $3$**  
   El tiempo de servicio en $3$ es $s'_3 = 4.0$. Por tanto, el vehículo termina en:
   $$17.00 + 4.00 = 21.00$$

3. **Desplazamiento hacia el nodo $5$**  
   Se parte en $t = 21.00$ y se llega al nodo $5$ en $t = 31.00$.

4. **Servicio en el nodo $5$ y finalización**  
   El tiempo de servicio en $5$ es $s'_5 = 3.0$. El instante final queda:
   $$31.00 + 3.00 = 34.00$$

Por definición, este último instante corresponde a la función objetivo:
$$\text{Objetivo} = 34.0$$

---

### Lectura global de la solución

El resultado muestra claramente la lógica integrada del problema:

- La ruta de reparación determina los tiempos $d_i$ (cuándo la red se vuelve utilizable en los nodos dañados).
- La ruta de ayuda se calcula respetando dichas disponibilidades, incorporando esperas cuando sea necesario.
- La función objetivo $34.0$ mide el desempeño global del operativo: viaje + posibles esperas + servicio, hasta completar el último punto de demanda.

En un contexto aplicado, $34.0$ puede interpretarse como el **tiempo total de respuesta** requerido para completar toda la operación de ayuda bajo restricciones de reparación.


## Caso de estudio 2: instancia del paper (Figura 3 y Tabla 1)

### Descripción de los datos y su interpretación práctica

En este segundo caso de estudio se considera la instancia propuesta en el artículo original, correspondiente a la Figura 3 y la Tabla 1 del paper. A diferencia del ejemplo ilustrativo anterior, este escenario presenta una red más compleja, con un mayor número de nodos de reparación y una estructura de conectividad más rica, lo que permite evaluar el comportamiento del algoritmo en una situación más realista.

El escenario se modela mediante un grafo no dirigido cuyos arcos representan los tiempos de desplazamiento entre ubicaciones. Todas las operaciones parten de un nodo depósito común, identificado como el nodo $0$, que actúa como origen tanto para el equipo de reparación como para el vehículo de ayuda humanitaria.

En esta instancia se consideran seis nodos de reparación, $\{2,3,5,7,9,10\}$, cada uno con un tiempo de reparación $s_i$ distinto. Estos valores reflejan diferentes niveles de daño en la infraestructura y determinan el orden y el impacto temporal de las reparaciones sobre la accesibilidad de la red. La finalización de estas reparaciones define de manera progresiva la disponibilidad de determinados caminos.

Por su parte, los nodos de demanda son $\{1,4,6\}$, todos ellos con un tiempo de servicio uniforme $s'_i = 2$. Estos nodos representan ubicaciones que requieren atención humanitaria una vez que las rutas necesarias para acceder a ellas se encuentran operativas.

La característica más relevante de este escenario es la fuerte dependencia entre ambos tipos de nodos. Muchos de los caminos que conectan el depósito con los nodos de demanda atraviesan nodos de reparación, lo que implica que el vehículo de ayuda puede verse obligado a esperar hasta que ciertas reparaciones hayan finalizado. Esta interacción hace que la planificación conjunta de ambas rutas sea esencial para minimizar el tiempo total de respuesta.


![Grafo correspondiente al ejemplo pequeño](IMAGENES/image2.png)

In [6]:
# --- EJEMPLO PAPER ---

def build_example_paper() -> ProblemData:
    adj = {}
    add_undirected_edge(adj, 0, 3, 5)
    add_undirected_edge(adj, 0, 8, 10)
    add_undirected_edge(adj, 0, 2, 6)
    add_undirected_edge(adj, 3, 4, 5)
    add_undirected_edge(adj, 4, 9, 8)
    add_undirected_edge(adj, 9, 8, 1)
    add_undirected_edge(adj, 8, 7, 5)
    add_undirected_edge(adj, 7, 6, 6)
    add_undirected_edge(adj, 6, 10, 3)
    add_undirected_edge(adj, 6, 5, 2)
    add_undirected_edge(adj, 5, 1, 2)
    add_undirected_edge(adj, 1, 2, 5)
    add_undirected_edge(adj, 2, 6, 7)

    return ProblemData(
        adj=adj,
        depot=0,
        repair_nodes={2, 3, 5, 7, 9, 10},
        demand_nodes={1, 4, 6},
        repair_time={2: 6, 3: 5, 5: 3, 7: 14, 9: 7, 10: 16},
        demand_service_time={1: 2, 4: 2, 6: 2},
    )

data = build_example_paper()
params = ACOParams(alpha=1.0, beta=2.0, rho=0.2, Q=1.0, num_ants=60, num_iters=300, seed=7)

aco = IOSRCRVD_ACO(data, params)
best = aco.solve()

print("=== EJEMPLO PAPER (Figura 3 / Tabla 1) ===")
print("Ruta reparación:", best.repair_route)
print("s_i:", {k: data.repair_time[k] for k in sorted(data.repair_nodes)})
print("d_i:", {k: round(best.repair_done_time[k], 2) for k in sorted(best.repair_done_time)})
print("Ruta ayuda:", best.relief_route)
print("s'_i:", {k: data.demand_service_time[k] for k in sorted(data.demand_nodes)})
print("Objetivo:", round(best.objective, 2))

print("--- Traza ayuda ---")
for a, b, ts, ta, sv, tf in trace_relief(data, best):
    print(f"{a}->{b}: sale {ts:.2f}, llega {ta:.2f}, servicio {sv:.2f}, termina {tf:.2f}")


=== EJEMPLO PAPER (Figura 3 / Tabla 1) ===
Ruta reparación: [0, 3, 2, 5, 7, 9, 10]
s_i: {2: 6, 3: 5, 5: 3, 7: 14, 9: 7, 10: 16}
d_i: {2: 27.0, 3: 10.0, 5: 37.0, 7: 59.0, 9: 72.0, 10: 103.0}
Ruta ayuda: [0, 4, 1, 6]
s'_i: {1: 2, 4: 2, 6: 2}
Objetivo: 46.0
--- Traza ayuda ---
0->4: sale 0.00, llega 15.00, servicio 2.00, termina 17.00
4->1: sale 17.00, llega 38.00, servicio 2.00, termina 40.00
1->6: sale 40.00, llega 44.00, servicio 2.00, termina 46.00


### Interpretación de la solución obtenida

La ejecución del algoritmo ACO sobre esta instancia produce una solución compuesta por dos rutas coordinadas y un valor de la función objetivo igual a **$46.0$**, que representa el instante en el que se completa el último servicio de ayuda.

La ruta obtenida para el equipo de reparación es:

- Ruta reparación: $[0, 3, 2, 5, 7, 9, 10]$

A partir de esta ruta se calculan los instantes de finalización de las reparaciones $d_i$:

- $d_3 = 10.0$
- $d_2 = 27.0$
- $d_5 = 37.0$
- $d_7 = 59.0$
- $d_9 = 72.0$
- $d_{10} = 103.0$

Cada valor $d_i$ indica el momento a partir del cual el nodo de reparación correspondiente queda completamente operativo. Estos tiempos determinan la evolución temporal de la red y condicionan directamente las decisiones del vehículo de ayuda.

La ruta obtenida para el vehículo de ayuda es:

- Ruta ayuda: $[0, 4, 1, 6]$

La traza temporal asociada a esta ruta es la siguiente:

- $0 \to 4$: sale $0.00$, llega $15.00$, servicio $2.00$, termina $17.00$
- $4 \to 1$: sale $17.00$, llega $38.00$, servicio $2.00$, termina $40.00$
- $1 \to 6$: sale $40.00$, llega $44.00$, servicio $2.00$, termina $46.00$

El vehículo de ayuda alcanza el nodo $4$ en el tiempo $15.00$ y completa el servicio en $17.00$. Posteriormente se desplaza al nodo $1$, al que llega en el tiempo $38.00$. Este valor de llegada incorpora de manera implícita posibles esperas causadas por reparaciones aún no finalizadas en los nodos intermedios del camino óptimo. Tras completar el servicio en el nodo $1$, el vehículo se dirige al nodo $6$, finalizando el último servicio en el tiempo $46.00$.

Por definición, este instante corresponde a la función objetivo:
$$
\text{Objetivo} = 46.0
$$

Desde una perspectiva global, la solución muestra cómo el algoritmo coordina la secuencia de reparaciones con la planificación de la distribución de ayuda. La ruta de reparación determina la disponibilidad progresiva de la red a través de los valores $d_i$, mientras que la ruta de ayuda se adapta a dicha disponibilidad, incorporando esperas cuando es necesario. El valor final de $46.0$ puede interpretarse como el tiempo total de respuesta del sistema para este escenario, proporcionando una medida clara del desempeño operativo de la solución obtenida.